In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [89.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 48.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=5ffa971a7dfb033fe062bdd328232545929e78bdee321fd60f895ba66ad2d7ec
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-13 17:08:53--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-08-13 17:08:54 (1.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Team-3-Project").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://team-3-s3-data-bucket.s3.amazonaws.com/202111-citibike-tripdata.csv"
spark.sparkContext.addFile(url)
bikedata_df = spark.read.csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
bikedata_df.head(5)

[Row(ride_id='F8C750DE34200458', rideable_type='electric_bike', started_at=datetime.datetime(2021, 11, 8, 7, 34, 45), ended_at=datetime.datetime(2021, 11, 8, 7, 52, 14), start_station_name='Franklin Ave & St Marks Ave', start_station_id='4107.05', end_station_name='Carroll St & Smith St', end_station_id='4225.14', start_lat=40.6758324, start_lng=-73.9561677, end_lat=40.680611, end_lng=-73.99475825, member_casual='member'),
 Row(ride_id='150B96183EA723BE', rideable_type='electric_bike', started_at=datetime.datetime(2021, 11, 24, 10, 27, 6), ended_at=datetime.datetime(2021, 11, 24, 10, 34, 40), start_station_name='Pleasant Ave & E 120 St', start_station_id='7579.01', end_station_name='Willis Ave & E 137 St', end_station_id='7726.02', start_lat=40.7974772, start_lng=-73.9311847, end_lat=40.808384, end_lng=-73.923604, member_casual='member'),
 Row(ride_id='55BA8CE0120B8994', rideable_type='electric_bike', started_at=datetime.datetime(2021, 11, 27, 18, 11, 36), ended_at=datetime.datetime(20

In [6]:
bikedata_df.show()


+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------+--------------+------------+--------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|  start_lat|     start_lng|     end_lat|       end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+-----------+--------------+------------+--------------+-------------+
|F8C750DE34200458|electric_bike|2021-11-08 07:34:45|2021-11-08 07:52:14|Franklin Ave & St...|         4107.05|Carroll St & Smit...|       4225.14| 40.6758324|   -73.9561677|   40.680611|  -73.99475825|       member|
|150B96183EA723BE|electric_bike|2021-11-24 10:27:06|2021-11-24 10:34:40|Pleasant Ave & E ...|         7579.01|Willis Ave & E 13...|     

In [7]:

new_bikedata_df = bikedata_df.drop('start_station_id', 'end_station_id')

In [8]:
new_bikedata_df.show()

+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|    end_station_name|  start_lat|     start_lng|     end_lat|       end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+--------------------+-----------+--------------+------------+--------------+-------------+
|F8C750DE34200458|electric_bike|2021-11-08 07:34:45|2021-11-08 07:52:14|Franklin Ave & St...|Carroll St & Smit...| 40.6758324|   -73.9561677|   40.680611|  -73.99475825|       member|
|150B96183EA723BE|electric_bike|2021-11-24 10:27:06|2021-11-24 10:34:40|Pleasant Ave & E ...|Willis Ave & E 13...| 40.7974772|   -73.9311847|   40.808384|    -73.923604|       member|
|55BA8CE0120B8994|electric_bike|2021-11-27 18:11:36|2021-11-27 19:19:16|West St 

In [9]:
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
import plotly.express as px
##import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import datetime as dt

ID=0


In [16]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import haversine as hs
from haversine import Unit
#!pip install haversine

new_bikedata_df.insert(12, 'distance_traveled',"0" ,  allow_duplicates = False)
new_bikedata_df.head(10)

#new_bikedata_df.insert(11, 'distance_traveled', dtype =int, hs.haversine(loc1,loc2, unit=Unit.MILES), allow_duplicates = False)
#new_bikedata_df['distance_traveled'] = hs.haversine(loc1,loc2,unit=Unit.MILES)

AttributeError: ignored

In [14]:
new_bikedata_df.columns

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'end_station_name',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual']

In [18]:
##Add new columns for POSIX timestamps

new_bikedata_df.assign(10, 'ended_at_sec', new_bikedata_df['ended_at'], allow_duplicates = False)
new_bikedata_df.assign(10, 'started_at_sec', new_bikedata_df['started_at'], allow_duplicates = False)

AttributeError: ignored

In [ ]:
# COLUMNS
new_bikedata_df.columns

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'end_station_name',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual']

In [15]:
# LIST DATAFRAME DATA TYPES
new_bikedata_df.dtypes

[('ride_id', 'string'),
 ('rideable_type', 'string'),
 ('started_at', 'timestamp'),
 ('ended_at', 'timestamp'),
 ('start_station_name', 'string'),
 ('end_station_name', 'string'),
 ('start_lat', 'double'),
 ('start_lng', 'double'),
 ('end_lat', 'double'),
 ('end_lng', 'double'),
 ('member_casual', 'string')]

In [ ]:
# FIND NULL VALUES
for column in new_bikedata_df.columns: print(f"Column {Column} has {new_bikedata_df[column].isnull().sum()}null values")

NameError: ignored

In [ ]:
# DROP NULL ROWS
new_bikedata_df = new_bikedata_df.dropna()

In [ ]:
# FIND DUPLICATE ENTRIES
print(f"Duplicate entries: {new_bikedata_df.duplicated().sum()}")

AttributeError: ignored

In [ ]:
# TRANSFORM STRING COLUMN
def change_string(rideable_type):
    if rideable_type == "electric_bike":
        return 1
    else:
        return 0
    
new_bikedata_df["rideable_type"] = new_bikedata_df["rideable_type"].apply(change_string)
new_bikedata_df.head(20)

TypeError: ignored

In [ ]:

def change_string(ride_id):
    if ride_id != "0":
        global ID
        ID = ID +1
        return ID
    else:
        
        return ID
    
new_bikedata_df["ride_id"] = new_bikedata_df["ride_id"].apply(change_string)
new_bikedata_df.head(20)

TypeError: ignored

In [ ]:
new_bikedata_df['rideable_type']=new_bikedata_df.rideable_type.astype('int16')
new_bikedata_df['start_station_name']=new_bikedata_df.start_station_name.astype('string')
new_bikedata_df['end_station_name']=new_bikedata_df.end_station_name.astype('string')
new_bikedata_df['member_casual']=new_bikedata_df.member_casual.astype('string')
new_bikedata_df['ride_id']=new_bikedata_df.ride_id.astype('int16')
new_bikedata_df['started_at']= pd.to_datetime(new_bikedata_df['started_at'])
new_bikedata_df['ended_at']= pd.to_datetime(new_bikedata_df['ended_at'])
## LIST DATAFRAME DATA TYPES
new_bikedata_df.dtypes
new_bikedata_df.loc[new_bikedata_df['rideable_type'] == 1].head(10)

In [ ]:
new_bikedata_df.loc.to_csv('202111-citibike-trpdata-electric.csv', index=False)
new_bikedata_df.loc[new_bikedata_df['rideable_type'] == 1].head(10)